# Audio Recording with PyAudio

In [1]:
import pyaudio

import wave

import keyboard

import threading

import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [13]:
#Params
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS_MAX = 60 # Set the maximum recording duration in seconds

### Instantiate PyAudio Object

In [11]:
p = pyaudio.PyAudio()

In [14]:
import time

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print(f"Recording will start in 3 seconds... (Press 's' to stop or maximum duration is {RECORD_SECONDS_MAX} seconds)")

time.sleep(3)

print(f"Recording... (Press 'Ctrl + C' to stop or maximum duration is {RECORD_SECONDS_MAX} seconds)")


frames = []

def stop_recording():
    global RECORD_SECONDS
    RECORD_SECONDS = RECORD_SECONDS_MAX


# Start a timer to stop recording after RECORD_SECONDS_MAX seconds
RECORD_SECONDS = 0
timer = threading.Timer(RECORD_SECONDS_MAX, stop_recording)
timer.start()


try:
    while RECORD_SECONDS < RECORD_SECONDS_MAX:
        data = stream.read(CHUNK, exception_on_overflow = False)
        frames.append(data)
except KeyboardInterrupt:
    # Ctrl+C was pressed, stop recording
    print("Finished recording.")
    pass



print("Finished recording.")


stream.stop_stream()
stream.close()
p.terminate()





WAVE_OUTPUT_FILENAME = "data/raw/test3.wav"
with wave.open(WAVE_OUTPUT_FILENAME, 'wb') as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))

Recording will start in 3 seconds... (Press 's' to stop or maximum duration is 60 seconds)
Recording... (Press 'Ctrl + C' to stop or maximum duration is 60 seconds)
Finished recording.


# Audio Transcription with Whisper

In [2]:
import openai
import os

PATH = os.getcwd()

audio_file = open(f"{PATH}/data/raw/test3.wav", "rb")

In [3]:
#OpenAI authorizations
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organizations = "org-cMwR4RMO7oQ1BYbT8J6l48ai"

In [5]:
#Transcript
transcript = openai.Audio.transcribe("whisper-1", audio_file)

In [8]:
#print output
transcript["text"]

"L'Intelligence Artificielle Générative, ou IAG, représente une avancée technologique majeure qui ouvre de nouvelles perspectives pour les startups en matière d'innovation et de créativité. 1. Création du contenu et design innovants. Les startups peuvent exploiter l'IAG pour créer du contenu de manière automatisée et innovante. 2. Personnalisation de l'expérience client. Grâce à l'IAG, les startups peuvent offrir une expérience client personnalisée et immersive. 3. Optimisation des processus de production. L'IAG permet aux startups de perfectionner leurs processus de production. Des villes comme Leucato, Cambrésie ou Athènes utilisent des modèles d'IAG pour optimiser la gestion des stocks, améliorer la planification de la chaîne d'approvisionnement et réduire les coûts de production. 4. Accélération de la recherche et du développement. Les startups œuvrant dans la recherche et le développement peuvent tirer parti de l'IAG pour offrir des solutions innovantes."

## Text transformation with GPT

In [5]:
# Defining helper function
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [17]:
import pandas as pd
models = openai.Model.list()
models_df = pd.DataFrame(models["data"])

In [22]:
models_df[models_df["id"] == 'gpt-3.5-turbo']

,id,object,created,owned_by,permission,root,parent
52,gpt-3.5-turbo,model,1677610602,openai,"[{'id': 'modelperm-ILldYg889l2lED1b99xWEfey', ...",gpt-3.5-turbo,None


### openai.ChatCompletion test

In [13]:
text = f"""L'Intelligence Artificielle Générative, ou IAG, \
représente une avancée technologique majeure qui ouvre de nouvelles\
perspectives pour les startups en matière d'innovation et de \
créativité. 1. Création du contenu et design innovants. Les startups peuvent \
exploiter l'IAG pour créer du contenu de manière automatisée et innovante. 2. Personnalisation \
de l'expérience client. Grâce à l'IAG, les startups peuvent offrir une expérience client personnalisée et \
immersive. 3. Optimisation des processus de production. L'IAG permet aux \
startups de perfectionner leurs processus de production. Des villes comme Leucato, \
Cambrésie ou Athènes utilisent des modèles d'IAG pour optimiser la gestion des stocks, améliorer \
la planification de la chaîne d'approvisionnement et réduire les coûts de production. 4. Accélération de la \
recherche et du développement. Les startups œuvrant dans la recherche et le développement peuvent tirer parti \
de l'IAG pour offrir des solutions innovantes."""
    
    
    
prompt = f"""You work for a web agency specializing in website creation and SEO. The clients send the articles they want to publish text that are audio files transcripted by a speech-to-text model. 
However this model is not perfect, and sometimes, the generated text is not a ready-to-published text.
Your role is to modify this text to make it ready to be published on the web.
The text you need to edit is delimited by triple backticks.

You remove filling words.
You segment the text into coherent parts when it is needed.
You add punctuation where it is needed.
You proofread and correct the text.
You explicit the abbreviations.
You edit key words so that they are in bold.

The text might contain a Title, subtitles and bullet points, but they are not likely to be explicitly announced. If you recognize a title, a subtitle or a list, edit the text format in consequences.

Format everything as HTML that can be used in a website. 
Place the edited text in a <div> element.

Your answer must be in the same language as the text.

```{text}```"""



In [14]:
response = get_completion(prompt)

In [15]:
from IPython.display import display, HTML
display(HTML(response))